In [1]:
from exp.utils import *
from exp.models import *
from exp.losses import *
from tqdm.notebook import tqdm
from multiprocessing import Pool
from PIL import Image

import torch
import torch.nn as NN
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

In [2]:
picked_labels = ["Atelectasis", "Cardiomegaly", "Pneumonia"]

In [5]:
architecture = "MNASNET1_v3"

In [6]:
#for label in picked_labels:
for label in ["Atelectasis"]:
    print(f"Computing metrics for '{label}'")
    
    seed = 92
    model_name = f"{architecture}_{label}"
    print(model_name)
    s = 224
    image_size = (s, s)
    labels = get_labels()
    bs = 16
    device = get_device()
    
    sigmoid = NN.Sigmoid()
    model = load_model(model_name)
    model.eval();
    model = model.to(device)
    
    seed_everything(seed=seed)
    
    train_df, _, test_df = get_dataframes(include_labels=labels, 
                                             small=False)

    train_df = get_binary_df(label, train_df)
    test_df = get_binary_df(label, test_df)

    train_label = train_df[[label]].values
    neg_weights, pos_weights = compute_class_freqs(train_label)
    neg_weights, pos_weights = torch.Tensor(neg_weights), torch.Tensor(pos_weights)

    _, test_tfs = get_transforms(image_size=image_size)

    test_ds  = CRX8_Data(test_df , get_image_path(), label, image_size=image_size, transforms=test_tfs)

    test_dl  = DataLoader(test_ds,  batch_size=bs, shuffle=False)

    criterion = get_weighted_loss_with_logits(pos_weights.to(device), 
                                              neg_weights.to(device))

    history = validate(model, 
             criterion, 
             test_dl, 
             model_name,
             device=device)
    
    tmp = history
    tmp["loss"] = tmp["loss"].mean()
    tmp = {k:[v] for k, v in tmp.items()}
    model_metrics = pd.DataFrame(tmp.values(), index=tmp.keys(), columns=[model_name])
    #save_metrics(model_metrics, model_name)
    
FERTIG()

Computing metrics for 'Atelectasis'
MNASNET1_v3_Atelectasis
Using the GPU!


KeyboardInterrupt: 

In [ ]:
model_metrics